In [333]:
import api
import importlib
import report
import model

In [564]:
importlib.reload(api)

<module 'api' from '/home/paul/git/druid-api/api.py'>

In [565]:
c = api.default_local_docker_config(8899)
#c.add_host("localhost", 8899)
d = api.Druid(c)

In [313]:
ncs = [nc for nc in d.node_configs.values()]
nc = ncs[0]
vars(nc)

{'host_name': '172.18.0.4',
 'port': 8081,
 'endpoint': '172.18.0.4:8081',
 'roles': ['coordinator', 'overlord'],
 'protocol': 'http:',
 'private_host': '172.18.0.4',
 'private_port': 8081,
 'private_endpoint': '172.18.0.4:8081',
 'tier': None,
 'prefix': None}

In [309]:
d.config_for_service(api.NodeConfig.COORDINATOR)

In [282]:
d.servers()

body: {'query': '\n          SELECT\n\n            "server", "server_type", "tier", "host", "plaintext_port",\n            "tls_port", "curr_size", "max_size"\n          FROM sys.servers   \n        '}
http://localhost:8899/druid/v2/sql


[{'server': '172.18.0.4:8081',
  'server_type': 'coordinator',
  'tier': None,
  'host': '172.18.0.4',
  'plaintext_port': 8081,
  'tls_port': -1,
  'curr_size': 0,
  'max_size': 0},
 {'server': '172.18.0.7:8083',
  'server_type': 'historical',
  'tier': '_default_tier',
  'host': '172.18.0.7',
  'plaintext_port': 8083,
  'tls_port': -1,
  'curr_size': 13254186,
  'max_size': 300000000000},
 {'server': '172.18.0.5:8082',
  'server_type': 'broker',
  'tier': None,
  'host': '172.18.0.5',
  'plaintext_port': 8082,
  'tls_port': -1,
  'curr_size': 0,
  'max_size': 0},
 {'server': '172.18.0.4:8081',
  'server_type': 'overlord',
  'tier': None,
  'host': '172.18.0.4',
  'plaintext_port': 8081,
  'tls_port': -1,
  'curr_size': 0,
  'max_size': 0},
 {'server': '172.18.0.8:8888',
  'server_type': 'router',
  'tier': None,
  'host': '172.18.0.8',
  'plaintext_port': 8888,
  'tls_port': -1,
  'curr_size': 0,
  'max_size': 0},
 {'server': '172.18.0.6:8091',
  'server_type': 'middle_manager',
  't

In [314]:
lu = d.lookups()

http://172.18.0.4:8081/druid/coordinator/v1/isLeader


In [315]:
lu.all_lookups()

http://172.18.0.4:8081/druid/coordinator/v1/lookups/config/all


{}

In [316]:
lu.tiers()

http://172.18.0.4:8081/druid/coordinator/v1/lookups/config


[]

In [203]:
lu.lookups_for_tier(api.DEFAULT_TIER)

http://localhost:8081/druid/coordinator/v1/lookups/config/_default_tier


NotFoundError: tier = _default_tier

In [198]:
lu.lookup(api.DEFAULT_TIER, "foo")

http://localhost:8081/druid/coordinator/v1/lookups/config/__default/foo


NotFoundError: tier = __default, lookup id = foo

In [162]:
lu.initialize()

http://localhost:8081/druid/coordinator/v1/lookups/config


In [165]:
tlu = api.map_lookup("1", {"true": "1", "false" : "0"})
tlu

{'version': '1',
 'lookupExtractorFactory': {'type': 'map',
  'injective': False,
  'map': {'true': '1', 'false': '0'}}}

In [204]:
lu.update(api.DEFAULT_TIER, "test", tlu)

body: {'version': '1', 'lookupExtractorFactory': {'type': 'map', 'injective': False, 'map': {'true': '1', 'false': '0'}}}
http://localhost:8081/druid/coordinator/v1/lookups/config/_default_tier



RestError: ('Request failed', 'http://localhost:8081/druid/coordinator/v1/lookups/config/_default_tier', 405)

In [208]:
h = d.lead_coordinator()

http://localhost:8081/druid/coordinator/v1/isLeader


In [209]:
h.servers()

http://localhost:8081/druid/coordinator/v1/servers


['172.18.0.7:8083']

In [193]:
h.servers()

http://localhost:8081/druid/coordinator/v1/servers


['172.18.0.7:8083']

In [194]:
h.server_details()

http://localhost:8081/druid/coordinator/v1/servers?simple=


[{'host': '172.18.0.7:8083',
  'tier': '_default_tier',
  'type': 'historical',
  'priority': 0,
  'currSize': 13254186,
  'maxSize': 300000000000}]

In [93]:
ds = d.datasource('wikiticker-2015-09-12-sampled')

http://localhost:8081/druid/coordinator/v1/isLeader


In [88]:
b = d.broker()
b.is_ready()

broker
http://localhost:8082/druid/broker/v1/loadstatus


True

In [94]:
ds.full_schema()

http://localhost:8082/druid/v2/datasources/wikiticker-2015-09-12-sampled?full=


{}

In [120]:
md = ds.metadata()

http://localhost:8081/druid/coordinator/v1/metadata/datasources/wikiticker-2015-09-12-sampled


In [123]:
n
for seg in md['segments']:
    n += seg['size']
n

13139688

In [332]:
md

{'name': 'wikiticker-2015-09-12-sampled',
 'properties': {'created': '2021-07-28T23:34:05.964Z'},
 'segments': [{'dataSource': 'wikiticker-2015-09-12-sampled',
   'interval': '2015-09-12T00:00:00.000Z/2015-09-12T01:00:00.000Z',
   'version': '2021-07-18T08:58:27.061Z',
   'loadSpec': {'type': 'local',
    'path': '/opt/shared/segments/wikiticker-2015-09-12-sampled/2015-09-12T00:00:00.000Z_2015-09-12T01:00:00.000Z/2021-07-18T08:58:27.061Z/0/index.zip'},
   'dimensions': 'added,channel,cityName,comment,countryIsoCode,countryName,deleted,delta,isAnonymous,isMinor,isNew,isRobot,isUnpatrolled,metroCode,namespace,page,regionIsoCode,regionName,user',
   'metrics': '',
   'shardSpec': {'type': 'numbered', 'partitionNum': 0, 'partitions': 1},
   'binaryVersion': 9,
   'size': 55706,
   'identifier': 'wikiticker-2015-09-12-sampled_2015-09-12T00:00:00.000Z_2015-09-12T01:00:00.000Z_2021-07-18T08:58:27.061Z'},
  {'dataSource': 'wikiticker-2015-09-12-sampled',
   'interval': '2015-09-12T01:00:00.000

In [519]:
'a,b'.split(',')

['a', 'b']

In [520]:
md['segments'][0]['dimensions'].split(',')

['added',
 'channel',
 'cityName',
 'comment',
 'countryIsoCode',
 'countryName',
 'deleted',
 'delta',
 'isAnonymous',
 'isMinor',
 'isNew',
 'isRobot',
 'isUnpatrolled',
 'metroCode',
 'namespace',
 'page',
 'regionIsoCode',
 'regionName',
 'user']

In [45]:
ds.audit_history()

http://localhost:8081/druid/coordinator/v1/rules/history?count=25


[]

In [21]:
ivs = ds.intervals()

http://localhost:8081/druid/coordinator/v1/datasources/wikiticker-2015-09-12-sampled/intervals


In [22]:
ivs[0:10]

['2015-09-12T23:00:00.000Z/2015-09-13T00:00:00.000Z',
 '2015-09-12T22:00:00.000Z/2015-09-12T23:00:00.000Z',
 '2015-09-12T21:00:00.000Z/2015-09-12T22:00:00.000Z',
 '2015-09-12T20:00:00.000Z/2015-09-12T21:00:00.000Z',
 '2015-09-12T19:00:00.000Z/2015-09-12T20:00:00.000Z',
 '2015-09-12T18:00:00.000Z/2015-09-12T19:00:00.000Z',
 '2015-09-12T17:00:00.000Z/2015-09-12T18:00:00.000Z',
 '2015-09-12T16:00:00.000Z/2015-09-12T17:00:00.000Z',
 '2015-09-12T15:00:00.000Z/2015-09-12T16:00:00.000Z',
 '2015-09-12T14:00:00.000Z/2015-09-12T15:00:00.000Z']

In [23]:
ds.interval_servers(ivs[0])

http://localhost:8081/druid/coordinator/v1/datasources/wikiticker-2015-09-12-sampled/intervals/2015-09-12T23:00:00.000Z_2015-09-13T00:00:00.000Z/serverview


[{'segment': {'dataSource': 'wikiticker-2015-09-12-sampled',
   'interval': '2015-09-12T23:00:00.000Z/2015-09-13T00:00:00.000Z',
   'version': '2021-07-18T08:58:27.061Z',
   'loadSpec': {'type': 'local',
    'path': '/opt/shared/segments/wikiticker-2015-09-12-sampled/2015-09-12T23:00:00.000Z_2015-09-13T00:00:00.000Z/2021-07-18T08:58:27.061Z/0/index.zip'},
   'dimensions': 'added,channel,cityName,comment,countryIsoCode,countryName,deleted,delta,isAnonymous,isMinor,isNew,isRobot,isUnpatrolled,metroCode,namespace,page,regionIsoCode,regionName,user',
   'metrics': '',
   'shardSpec': {'type': 'numbered', 'partitionNum': 0, 'partitions': 1},
   'binaryVersion': 9,
   'size': 255528,
   'identifier': 'wikiticker-2015-09-12-sampled_2015-09-12T23:00:00.000Z_2015-09-13T00:00:00.000Z_2021-07-18T08:58:27.061Z'},
  'servers': [{'name': '172.18.0.6:8083',
    'host': '172.18.0.6:8083',
    'hostAndTlsPort': None,
    'maxSize': 300000000000,
    'type': 'historical',
    'tier': '_default_tier',
  

In [101]:
import pydruid.client as client

In [102]:
query = client.PyDruid("http://localhost:8899", 'druid/v2')

In [115]:
result = query.segment_metadata(datasource="wikiticker-2015-09-12-sampled", merge=True,
                               analysisTypes=["cardinality", "interval", "minmax", "size"])

In [322]:
vars(result)

{'query_dict': {'queryType': 'segmentMetadata',
  'dataSource': 'wikiticker-2015-09-12-sampled',
  'merge': True,
  'analysisTypes': ['cardinality', 'interval', 'minmax', 'size']},
 'query_type': 'segmentMetadata',
 'result': [{'id': 'merged',
   'intervals': ['2015-09-12T00:00:00.000Z/2015-09-13T00:00:00.000Z'],
   'columns': {'__time': {'type': 'LONG',
     'hasMultipleValues': False,
     'hasNulls': False,
     'size': 392440,
     'cardinality': None,
     'minValue': None,
     'maxValue': None,
     'errorMessage': None},
    'added': {'type': 'LONG',
     'hasMultipleValues': False,
     'hasNulls': False,
     'size': 313952,
     'cardinality': None,
     'minValue': None,
     'maxValue': None,
     'errorMessage': None},
    'channel': {'type': 'STRING',
     'hasMultipleValues': False,
     'hasNulls': False,
     'size': 510330,
     'cardinality': 45,
     'minValue': '#ar.wikipedia',
     'maxValue': '#zh.wikipedia',
     'errorMessage': None},
    'cityName': {'type': 

In [329]:
ar = result.result[0]

In [119]:
n = 0
for c in x['columns'].values():
    n += c['size']
n

6410541

In [529]:
importlib.reload(report)

<module 'report' from '/home/paul/git/druid-api/report.py'>

In [530]:
report.styles()

In [414]:
report.show_table(["a", "b"], [[10, "foo"]], ["druid-key", "druid-value"])

a,b
10,foo


In [394]:
d.lead_coordinator().is_healthy()

http://172.18.0.4:8081/druid/coordinator/v1/isLeader
http://172.18.0.4:8081/status/health


True

In [566]:
importlib.reload(model)

<module 'model' from '/home/paul/git/druid-api/model.py'>

In [560]:
m = model.table_model(d, 'wikiticker-2015-09-12-sampled')

http://172.18.0.4:8081/druid/coordinator/v1/datasources/wikiticker-2015-09-12-sampled/intervals
http://172.18.0.4:8081/druid/coordinator/v1/metadata/datasources/wikiticker-2015-09-12-sampled


In [573]:
importlib.reload(report)

<module 'report' from '/home/paul/git/druid-api/report.py'>

In [561]:
report.table_report(m)

Property,Value
Name,wikiticker-2015-09-12-sampled
Created,2021-08-03T19:46:59.673Z
Records,39K
Size,7195 Kib
Column Size,6411 Kib
Non-Column Size,785 Kib
Row Width,183
Segments,24
Segment Total,6729 Kib
Non-Segment Size,466 Kib


In [574]:
r = report.Reports(d)

In [575]:
r.tables()

Table Name
wikipedia
wikiticker-2015-09-12-sampled


In [576]:
t = r.table('wikiticker-2015-09-12-sampled')

In [577]:
t.schema()

In [439]:
report.round_bin(123)

'123 Bytes'

In [442]:
%%html
<h4>foo</h4>

In [463]:
"FOO".lower()

'foo'

In [472]:
str(None)

'None'